In [4]:
import numpy as np
import math 

In [12]:
class RNN:

    def __init__(self, input_size, hidden_size, output_size) :
        
        self.input_size= input_size
        self.hidden_size= hidden_size
        self.output_size = output_size

        self.layers = self.init_params()

    def init_params(self):

        np.random.seed(0)

        # scaling factor 
        k= 1/math.sqrt(self.hidden_size)
        # Initialize weights and biases
        i_weight = np.random.rand(self.input_size,self.hidden_size)*2*k-k 
        h_weight = np.random.rand(self.hidden_size, self.hidden_size) * 2 * k - k
        h_bias = np.random.rand(1, self.hidden_size) * 2 * k - k
        o_weight = np.random.rand(self.hidden_size, self.output_size) * 2 * k - k

        o_bias = np.random.rand(1, self.output_size) * 2 * k - k
        return [i_weight, h_weight, h_bias, o_weight, o_bias]
    
    def forward(self, x):
        i_weight, h_weight, h_bias, o_bias = self.layers
        hidden = np.zeros((len(x),self.hidden_size))
        output=  np.zeros((len(x),self.output_size))

        
        for j in range(len(x)):
            # Compute input to hidden layer
            input_x = x[j] @ i_weight
            # Compute hidden state (using previous hidden state)
            hidden_x = input_x + hidden[max(j-1,0)] @ h_weight + h_bias
            # Apply tanh activation function
            hidden_x = np.tanh(hidden_x)
            hidden[j] = hidden_x
            # Compute output
            output_x = hidden_x @ o_weight + o_bias
            # Apply softmax to get probabilities
            output[j] = np.exp(output_x) / np.sum(np.exp(output_x))
        
        return hidden, output
    
    def backward(self, x, grad, hidden, lr):
        i_weight, h_weight, h_bias, o_weight, o_bias = self.layers
        next_h_grad = None
        i_weight_grad, h_weight_grad, h_bias_grad, o_weight_grad, o_bias_grad = [0] * 5 

        for j in range(len(x)-1,-1,-1):
            out_grad = grad[j][np.newaxis, :]
            o_weight_grad += hidden[j][np.newaxis, :].T @ out_grad
            o_bias_grad += out_grad
            h_grad = out_grad @ o_weight.T

            if j<len(x)-1:
                hh_grad= next_h_grad @ h_weight.T
                h_grad += h_grad 

            tanh_deriv = 1- hidden[j][np.newaxis,:] **2
            h_grad = np.multiply(h_grad, tanh_deriv)
            next_h_grad = h_grad.copy()

            if j > 0:
                h_weight_grad += hidden[j-1][np.newaxis, :].T @ h_grad
                h_bias_grad += h_grad
            i_weight_grad += x[j][np.newaxis, :].T @ h_grad

            lr= lr/len(x)
            i_weight -=i_weight_grad * lr
            h_weight -= i_weight_grad * lr
            h_bias -= h_bias_grad*lr
            o_weight -= o_weight_grad * lr
            o_bias -= o_bias_grad * lr

            self.layers = [i_weight,h_weight,h_bias,o_weight, o_bias]
            


In [1]:
s = "([{}])"
s = s.replace('()', '').replace('[]', '').replace('{}', '')
s

'([])'

In [2]:
!pip install datasets


   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   -- ------------------------------------- 1.8/25.1 MB 10.1 MB/s eta 0:00:03
   ------- -------------------------------- 4.7/25.1 MB 11.9 MB/s eta 0:00:02
   ----------- ---------------------------- 7.3/25.1 MB 12.2 MB/s eta 0:00:02
   ------------- -------------------------- 8.4/25.1 MB 12.4 MB/s eta 0:00:02
   ------------- -------------------------- 8.7/25.1 MB 8.5 MB/s eta 0:00:02
   ---------------------- ----------------- 14.4/25.1 MB 11.8 MB/s eta 0:00:01
   -------------------------- ------------- 16.8/25.1 MB 11.9 MB/s eta 0:00:01
   ----------------------------- ---------- 18.6/25.1 MB 12.0 MB/s eta 0:00:01
   ----------------------------- ---------- 18.6/25.1 MB 12.0 MB/s eta 0:00:01
   ----------------------------- ---------- 18.6/25.1 MB 12.0 MB/s eta 0:00:01
   ----------------------------- ---------- 18.6/25.1 MB 12.0 MB/s eta 0:00:01
   ------------------------------------ --- 23.1/25.1 MB 9.5 MB/s e

In [ ]:
from datasets import load_dataset

# Load the dataset with streaming to avoid downloading the entire dataset
dataset = load_dataset("riotu-lab/ARABIC-RAW-TEXT", split='train', streaming=True)

# Extract the first 10,000 rows
sample_data = list(dataset.take(10000))

# Save the sample data to a text file
with open("arabic_raw_sample.txt", "w", encoding="utf-8") as f:
    for row in sample_data:
        f.write(row['text'] + "\n")

print("Sample data saved successfully.")
